In [10]:
import os
import pickle
import pydicom
import pandas as pd
root_folder_path  = "/home/single3/Documents/tintrung/brainmri/trungdc/1.2.840.113619.6.408.217046082646534226129609417598798276891/DICOM"

In [2]:
file_paths = []
for folder_name in os.listdir(root_folder_path):
    folder_path = os.path.join(root_folder_path, folder_name)
    for file_name in os.listdir(folder_path):
        file_paths.append(os.path.join(folder_path, file_name))
print(f"Total images: {len(file_paths)}")

Total images: 71852


In [5]:
df = pd.read_pickle("summary_dicom.pkl")
pd.set_option("display.max_colwidth", None)
df.head()

,StudyInstanceUID,SeriesInstanceUID,SeriesDescription,Rows,Columns,SliceThickness,PixelSpacing,SpacingBetweenSlices,SliceLocation,SmallestImagePixelValue,LargestImagePixelValue,WindowCenter,WindowWidth,ImagePositionPatient,ImageOrientationPatient,RepetitionTime,EchoTime,PatientPosition,NameFileXML
0,1.2.840.113619.6.388.101774739834280643528361334251926593124,1.2.840.113619.2.5.1916222523715996118091599611809219000,Apparent Diffusion Coefficient (mm2/s),256,256,5.0,"[1.0156, 1.0156]",6.0,12.219006,269,5118,1300.0,2400.0,"[-129.19, -128.506, 19.0657]","[0.999995, 0.00266694, 0.0014183, -0.00258601, 0.998522, -0.0542916]",4384.0,101.9,HFS,1.2.840.113619.2.311.118469111826723530957815646435435346224.dcm
1,1.2.840.113619.6.388.101774739834280643528361334251926593124,1.2.840.113619.2.5.1916222523715996118091599611809219000,Apparent Diffusion Coefficient (mm2/s),256,256,5.0,"[1.0156, 1.0156]",6.0,-53.683617,48,5624,1300.0,2400.0,"[-129.086, -132.089, -46.8369]","[0.999995, 0.00266694, 0.0014183, -0.00258601, 0.998522, -0.0542916]",4384.0,101.9,HFS,1.2.840.113619.2.311.125282295065266504184867863002451722448.dcm
2,1.2.840.113619.6.388.101774739834280643528361334251926593124,1.2.840.113619.2.5.1916222523715996118091599611809219000,Apparent Diffusion Coefficient (mm2/s),256,256,5.0,"[1.0156, 1.0156]",6.0,30.19245,346,4091,1300.0,2400.0,"[-129.218, -127.529, 37.0391]","[0.999995, 0.00266694, 0.0014183, -0.00258601, 0.998522, -0.0542916]",4384.0,101.9,HFS,1.2.840.113619.2.311.13734497280751956864742956698508702383.dcm
3,1.2.840.113619.6.388.101774739834280643528361334251926593124,1.2.840.113619.2.5.1916222523715996118091599611809219000,Apparent Diffusion Coefficient (mm2/s),256,256,5.0,"[1.0156, 1.0156]",6.0,-29.71903,205,4953,1300.0,2400.0,"[-129.124, -130.786, -22.8723]","[0.999995, 0.00266694, 0.0014183, -0.00258601, 0.998522, -0.0542916]",4384.0,101.9,HFS,1.2.840.113619.2.311.149759259314721767500053971461046076203.dcm
4,1.2.840.113619.6.388.101774739834280643528361334251926593124,1.2.840.113619.2.5.1916222523715996118091599611809219000,Apparent Diffusion Coefficient (mm2/s),256,256,5.0,"[1.0156, 1.0156]",6.0,0.23671,250,5017,1300.0,2400.0,"[-129.171, -129.158, 7.08339]","[0.999995, 0.00266694, 0.00141831, -0.00258601, 0.998522, -0.0542916]",4384.0,101.9,HFS,1.2.840.113619.2.311.162064945887768070496287289213718815545.dcm


In [6]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import matplotlib.pyplot as plt
%matplotlib inline


def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [14]:
x= read_xray(file_paths[0])
np.shape(x)[0]



256

In [22]:
height = []
width = []
for index, row in df.iterrows():
    cur = 0
    try:
        path = os.path.join(root_folder_path, row["StudyInstanceUID"])
        path = os.path.join(path, row["NameFileXML"])
        x = np.shape(read_xray(path))
        height.append(x[0])
        width.append(x[1])
    except:
        cur += 1



<ipython-input-6-36693932b096>:23: RuntimeWarning: invalid value encountered in true_divide
  data = data / np.max(data)


In [23]:
print(cur)

0


In [25]:
print(len(df))
print(len(height))

71852
71848
